### stat-RISMED program
Objective: To build a classification model to identify whether overseas regulatory action has been taken for a substandard medicine issue, based on the case description.
* Output of model will be used as input for stat-RISMED

### Setup

In [1]:
import re
import json
import pickle
import string
import numpy as np
import pandas as pd
import os

pd.options.mode.chained_assignment = None

In [35]:
import tensorflow as tf
import keras
from transformers import BertTokenizer, TFAutoModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, roc_auc_score, f1_score, recall_score, precision_score

In [3]:
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\desmondteoch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load and prepare data

In [4]:
# load full data
os.chdir(r"C:\Users\desmondteoch\Documents\CVU\Capstone\Impact Score")
data = pd.read_csv("Impact Score_2011tillJun2021_updated28Aug2021.csv", encoding="ISO-8859-1")

In [5]:
# select required columns only
overseas = data[["case_no","date_of_receipt","created_time","case_title","case_description",\
                "rismed_defect","rismed_defect_severity","rismed_overseas_action"]]
overseas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 893 entries, 0 to 892
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   case_no                 893 non-null    object
 1   date_of_receipt         893 non-null    object
 2   created_time            893 non-null    object
 3   case_title              893 non-null    object
 4   case_description        893 non-null    object
 5   rismed_defect           893 non-null    object
 6   rismed_defect_severity  893 non-null    object
 7   rismed_overseas_action  893 non-null    object
dtypes: object(8)
memory usage: 55.9+ KB


In [6]:
# recode "rismed_overseas_action" -- combine "Yes" and "Unknown" into one
overseas.loc[:, "label"] = overseas["rismed_overseas_action"].replace({"Yes": "Yes", "Unknown": "Yes",\
                                                                       "No": "No"})
le = LabelEncoder()
overseas.loc[:, "label"] = le.fit_transform(overseas["label"])
print("Class distribution:\n",overseas.label.value_counts())
# 1: "Yes", 0: "No"

Class distribution:
 label
1    569
0    324
Name: count, dtype: int64


In [7]:
overseas.loc[:, "text"] = overseas["case_description"]
overseas.head()

,case_no,date_of_receipt,created_time,case_title,case_description,rismed_defect,rismed_defect_severity,rismed_overseas_action,label,text
0,SVS2011-000102,25/2/2009,2/6/2011,Certain batches of Hospira's Desferrioxamine m...,Certain batches of Hospira's Desferrioxamine m...,Product physical issue,Low,Yes,1,Certain batches of Hospira's Desferrioxamine m...
1,SVS2011-000354,10/3/2011,13/6/2011,NUH found some batches of Adrenaline 1:1000 in...,NUH found some batches of adrenaline 1:1000 in...,"Product expiration date missing, illegible or ...",Medium,Unknown,1,NUH found some batches of adrenaline 1:1000 in...
2,SVS2011-000368,19/4/2011,1/7/2011,Dimenate tablets failed specification for assay,Drug Houses of Australia (DHA) informed HSA th...,Out of specification or out of trend test result,Medium,Yes,1,Drug Houses of Australia (DHA) informed HSA th...
3,SVS2011-000374,16/11/2010,4/7/2011,Recall of specific lots of Ebetrexat 100mg/ml ...,EB had received two separate alerts from Austr...,Contamination with glass and/or metal particle,High,Yes,1,EB had received two separate alerts from Austr...
4,SVS2011-000381,5/1/2011,4/7/2011,Hundreds in UK become pregnant despite contrac...,Nearly 600 women in UK who used a popular cont...,Lack of efficacy,Medium,No,0,Nearly 600 women in UK who used a popular cont...


In [8]:
def text_clean(context):
    clean_lst = ["\r","\n","   ",'"',"'s",'\d']
    punctuation_signs = list(".?:!,;/\{[}]@#$%^&*(|)-�")
    for x in clean_lst:
        context = context.str.replace(x,'')
    for punct in punctuation_signs:
        context = context.str.replace(punct,' ')
    context = context.str.lower()
    return context

def stop_words(context):
    stop_words = list(stopwords.words('english'))
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        context = context.str.replace(regex_stopword, '')
    for word in filter_words:
        regex_word = r"\b" + word + r"\b"
        context = context.str.replace(regex_word, '')
    return context
    
# filter_words = ['the','fda','us','usfda','hk','doh','ema','tga','taiwan','dh','es','pdr','canada','hsa','china','malaysia','singapore','germany','mg',
#                'ml','product','products','nbsp', 'warning letter','mah','company','please','uk']
filter_words = []

In [9]:
overseas.loc[:, "text"] = text_clean(overseas["text"])
overseas.loc[:, "text"] = stop_words(overseas["text"])

In [10]:
# word count
from itertools import chain
unique_terms = overseas.text.str.split()
unique_terms = set(list(chain(*unique_terms)))
print('Count of unique terms:',len(unique_terms))

Count of unique terms: 16859


### Load BioBERT

In [11]:
# Load BioBERT (discharge summaries)
path = r"C:\Users\desmondteoch\Documents\CVU\Capstone\biobert_pretrain_output_disch_100000"
tokenizer = BertTokenizer.from_pretrained(path, local_files_only=True)
bert = TFAutoModel.from_pretrained(path, from_pt=True)
vocab_file = '/biobert_pretrain_output_disch_100000/vocab.txt'

C:\Users\desmondteoch\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

### Split data for training

In [12]:
# extract 'text' and 'label' columns
X = overseas['text']
y = overseas['label']

In [13]:
# split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

# further split training into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=24)

print(f"Training data: {X_train.shape}\nValidation data: {X_val.shape}\nTesting data: {X_test.shape}")

Training data: (571,)
Validation data: (143,)
Testing data: (179,)


In [14]:
do_lower_case = True # use model for lowercase
max_len = 512

def bert_encode(texts, tokenizer, max_len=max_len):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [15]:
# convert training and test sets into keras readable format
inputs_train = bert_encode(X_train, tokenizer, max_len=max_len)
inputs_val = bert_encode(X_val, tokenizer, max_len=max_len)
inputs_test = bert_encode(X_test, tokenizer, max_len=max_len)
labels_train = y_train.values.astype(int)
labels_val = y_val.values.astype(int)
labels_test = y_test.values.astype(int)

In [18]:
# model architecture

def build_model(bert_layer, max_len=max_len):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    b = bert_layer([input_word_ids, input_mask, segment_ids])
    net = b.last_hidden_state
    net = tf.keras.layers.SpatialDropout1D(0.2)(net)
    net = tf.keras.layers.GlobalMaxPooling1D()(net)
    out = tf.keras.layers.Dense(1, activation='sigmoid')(net)  # 1 output node and sigmoid activation for binary classification
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    
    # Learning rate schedule
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-5,
                                                                decay_steps=10000,
                                                                decay_rate=0.9)
    
    model.compile(tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                  loss='binary_crossentropy',  # for binary classification
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(),
                           tf.keras.metrics.Recall(),
                           tf.keras.metrics.AUC()]
                 )
    
    # Early stopping if no further improvements during training
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=3,
                                                      restore_best_weights=True)
    
    model._name = 'BioBERT_Binary_Model'
    
    return model, early_stopping

In [19]:
## un-hash lines below to train the model
# Warning: takes a long time to run. load final model below, unless re-training is required.
# hyperparameter tuning
num_epochs = [3, 5]
batch_sizes = [8, 16]

# store metrics for each set of hyperparameters
metrics = {}

for epoch in num_epochs:
    for batch_size in batch_sizes:
        # build model
        model, early_stopping = build_model(bert, max_len=max_len)
        
        # train model
        train_history = model.fit(
            inputs_train, 
            labels_train,
            validation_data=(inputs_val, labels_val),
            epochs=epoch,
            batch_size=batch_size,
            callbacks=[early_stopping],
            verbose=1
        )
        
        # evaluate model on validation set
        evaluation = model.evaluate(inputs_val, labels_val)
        
        # add metrics to dict
        metrics[(epoch, batch_size)] = evaluation
        
        # print progress
        print(f"Metrics for epoch={epoch}, batch_size={batch_size}: {evaluation}.")
        
        ## save parameters
        model.save(f"./models/BioBERT_epoch_{epoch}_batchsize_{batch_size}")
        
        ## save model weights
        weights_path = f"./models/BioBERT_epoch_{epoch}_batchsize_{batch_size}_weights.h5"
        model.save_weights(weights_path)
        
        # plot loss vs epoch curve
        plt.figure(figsize=(10, 6))
        plt.plot(train_history.history['loss'])
        plt.plot(train_history.history['val_loss'])
        plt.title('Model loss')
        plt.xlabel('Epoch')
        plt.xticks(range(0,epoch))
        plt.ylabel('Loss')
        plt.legend(['Train', 'Validation'], loc='upper right')
        plt.savefig(f'./images/loss_curve_epoch_{epoch}_batchsize_{batch_size}.png')
        plt.close()

Epoch 1/3
72/72 [==============================] - 1176s 16s/step - loss: 0.7279 - accuracy: 0.5902 - precision_1: 0.6453 - recall_1: 0.7443 - auc_1: 0.5453 - val_loss: 0.6308 - val_accuracy: 0.6434 - val_precision_1: 0.6454 - val_recall_1: 0.9891 - val_auc_1: 0.6530
Epoch 2/3
72/72 [==============================] - 1158s 16s/step - loss: 0.6393 - accuracy: 0.6252 - precision_1: 0.6590 - recall_1: 0.8125 - auc_1: 0.6488 - val_loss: 0.6076 - val_accuracy: 0.6643 - val_precision_1: 0.6719 - val_recall_1: 0.9348 - val_auc_1: 0.7002
Epoch 3/3
5/5 [==============================] - 74s 14s/step - loss: 0.5939 - accuracy: 0.6643 - precision_1: 0.6964 - recall_1: 0.8478 - auc_1: 0.7367
Metrics for epoch=3, batch_size=8: [0.5939305424690247, 0.6643356680870056, 0.6964285969734192, 0.8478260636329651, 0.7366794347763062].
INFO:tensorflow:Assets written to: ./models/BioBERT_epoch_3_batchsize_8\assets


INFO:tensorflow:Assets written to: ./models/BioBERT_epoch_3_batchsize_8\assets


Epoch 1/3


36/36 [==============================] - 2004s 55s/step - loss: 0.6961 - accuracy: 0.5832 - precision_2: 0.6301 - recall_2: 0.7841 - auc_2: 0.5604 - val_loss: 0.6257 - val_accuracy: 0.6853 - val_precision_2: 0.6767 - val_recall_2: 0.9783 - val_auc_2: 0.6566
Epoch 2/3
36/36 [==============================] - 2025s 56s/step - loss: 0.6385 - accuracy: 0.6322 - precision_2: 0.6644 - recall_2: 0.8153 - auc_2: 0.6478 - val_loss: 0.5948 - val_accuracy: 0.6713 - val_precision_2: 0.6744 - val_recall_2: 0.9457 - val_auc_2: 0.7052
Epoch 3/3
5/5 [==============================] - 74s 14s/step - loss: 0.5726 - accuracy: 0.6853 - precision_2: 0.6942 - recall_2: 0.9130 - auc_2: 0.7290
Metrics for epoch=3, batch_size=16: [0.5725988745689392, 0.6853147149085999, 0.6942148804664612, 0.9130434989929199, 0.729006826877594].
INFO:tensorflow:Assets written to: ./models/BioBERT_epoch_3_batchsize_16\assets


INFO:tensorflow:Assets written to: ./models/BioBERT_epoch_3_batchsize_16\assets


...

72/72 [==============================] - 1202s 16s/step - loss: 0.7024 - accuracy: 0.5779 - precision_3: 0.6270 - recall_3: 0.7784 - auc_3: 0.5528 - val_loss: 0.5890 - val_accuracy: 0.6783 - val_precision_3: 0.6716 - val_recall_3: 0.9783 - val_auc_3: 0.7255
Epoch 2/5
72/72 [==============================] - 1170s 16s/step - loss: 0.6115 - accuracy: 0.6602 - precision_3: 0.6985 - recall_3: 0.7898 - auc_3: 0.6868 - val_loss: 0.5684 - val_accuracy: 0.6783 - val_precision_3: 0.7500 - val_recall_3: 0.7500 - val_auc_3: 0.7436
Epoch 3/5
72/72 [==============================] - 2601s 36s/step - loss: 0.4943 - accuracy: 0.7671 - precision_3: 0.7874 - recall_3: 0.8523 - auc_3: 0.8252 - val_loss: 0.6071 - val_accuracy: 0.6923 - val_precision_3: 0.6875 - val_recall_3: 0.9565 - val_auc_3: 0.7757
Epoch 4/5
72/72 [==============================] - 2632s 37s/step - loss: 0.4440 - accuracy: 0.7793 - precision_3: 0.8122 - recall_3: 0.8352 - auc_3: 0.8664 - val_loss: 0.5456 - val_accuracy: 0.6853 - val_p

INFO:tensorflow:Assets written to: ./models/BioBERT_epoch_5_batchsize_8\assets


Epoch 1/5


36/36 [==============================] - 2087s 57s/step - loss: 0.4810 - accuracy: 0.7653 - precision_4: 0.7946 - recall_4: 0.8352 - auc_4: 0.8391 - val_loss: 0.5427 - val_accuracy: 0.7133 - val_precision_4: 0.8072 - val_recall_4: 0.7283 - val_auc_4: 0.7991
Epoch 2/5
36/36 [==============================] - 2044s 57s/step - loss: 0.3473 - accuracy: 0.8511 - precision_4: 0.8825 - recall_4: 0.8750 - auc_4: 0.9212 - val_loss: 0.6177 - val_accuracy: 0.7203 - val_precision_4: 0.7241 - val_recall_4: 0.9130 - val_auc_4: 0.7931
Epoch 3/5
36/36 [==============================] - 2090s 58s/step - loss: 0.2372 - accuracy: 0.9037 - precision_4: 0.9160 - recall_4: 0.9290 - auc_4: 0.9649 - val_loss: 0.6223 - val_accuracy: 0.6923 - val_precision_4: 0.7927 - val_recall_4: 0.7065 - val_auc_4: 0.8047
Epoch 4/5
5/5 [==============================] - 78s 15s/step - loss: 0.5427 - accuracy: 0.7133 - precision_4: 0.8072 - recall_4: 0.7283 - auc_4: 0.7991
Metrics for epoch=5, batch_size=16: [0.54266279935836

INFO:tensorflow:Assets written to: ./models/BioBERT_epoch_5_batchsize_16\assets


In [20]:
# print best hyperparameters i.e. lowest validation loss
best_hyperparameters = min(metrics, key=lambda x: metrics[x][1])
print(f"Best Hyperparameters: {best_hyperparameters}, Metrics: {metrics[best_hyperparameters]}")

Best Hyperparameters: (3, 8), Metrics: [0.5939305424690247, 0.6643356680870056, 0.6964285969734192, 0.8478260636329651, 0.7366794347763062]


In [21]:
metrics

{(3, 8): [0.5939305424690247,
  0.6643356680870056,
  0.6964285969734192,
  0.8478260636329651,
  0.7366794347763062],
 (3, 16): [0.5725988745689392,
  0.6853147149085999,
  0.6942148804664612,
  0.9130434989929199,
  0.729006826877594],
 (5, 8): [0.5169349312782288,
  0.7412587404251099,
  0.8021978139877319,
  0.79347825050354,
  0.802642822265625],
 (5, 16): [0.5426627993583679,
  0.7132866978645325,
  0.8072289228439331,
  0.72826087474823,
  0.799126148223877]}

In [23]:
# load specific model for testing
epoch = 3
batch_size = 8

# model = keras.models.load_model(f'./models/BioBERT_BiLSTM_epoch_{epoch}_batchsize_{batch_size}')
model.load_weights(f'./models/BioBERT_epoch_{epoch}_batchsize_{batch_size}_weights.h5')

In [24]:
# predict on unseen data - testing set
model_pred = model.predict(inputs_test)
threshold = 0.5
# pred_test = test_set['pred']

6/6 [==============================] - 89s 14s/step


In [31]:
# classification report
model_pred_binary = (model_pred >= threshold).astype(int)
print(classification_report(labels_test, model_pred_binary))

              precision    recall  f1-score   support

           0       0.50      0.41      0.45        54
           1       0.76      0.82      0.79       125

    accuracy                           0.70       179
   macro avg       0.63      0.62      0.62       179
weighted avg       0.68      0.70      0.69       179



In [32]:
print(confusion_matrix(labels_test, model_pred_binary))
#[[TN  FP]
# [FN  TP]]

[[ 22  32]
 [ 22 103]]


In [36]:
# Get AUC score
model_pred_proba = model.predict(inputs_test)
auc = roc_auc_score(labels_test, model_pred_proba)

print("AUC:", auc)

6/6 [==============================] - 194s 32s/step
AUC: 0.6859259259259259
